## Tipo Abstrato de Dado (TAD) árvore Rubro Negra (RedBlack)

O código abaixo ilustra uma árvore Red-Black (Rubro Negra).

Uma árvore Red Black é uma árvore binária de busca com as 5 propriedades abaixo:

1. todo nó ou é preto ou é vermelho
2. a raiz é preta. 
3. todo nó folha é preto (e NULO)
4. se um nó é vermelho seus dois filhos são pretos 
5. todo caminho de um nó até um nó externo contém o mesmo número de nós pretos

Esta implementação não usa explicitamente os nós nulos. Isto apresenta vantagem em relação ao número de nós criados. Entretanto, a implementação é mais complexa, pois é necessário incluir diversas checagens adicionais nas funções de inclusão/exclusão/ajustes/sucessor.

Uma implementação utilizando os nós nulos explícitos está disponível no link: https://github.com/Marcosddf/algoritmoseestruturasdedados/blob/master/arvore_red_black-nos-nulos.ipynb


In [1]:
#include <iostream>
#include <stdlib.h> 
using namespace std;

In [2]:
enum Cor {
    VERMELHO = 1, PRETO = 2
}

Estrutura para uma árvore binária simples.
A árvore tem ponteiro para o nó pai.

In [3]:
struct tNo {
    int chave; // pode ser modificado paraqualquer tipo de dado
    tNo *esq, *dir, *pai;
    Cor cor;
};


Função para ajustar ponteiro _pai_ da árvore binária, usada na função de exclusão.

OBS.: As duas funções abaixo não fazem parte da implementação padrão da árvore red black, mas auxiliar na programção.

In [4]:
void ajustaNoPai(tNo *no, tNo *novo){
    if (no->pai != NULL) {
        if (no->pai->esq == no)                                
            no->pai->esq = novo;
        else 
            no->pai->dir = novo;
        if (novo != NULL)
           novo->pai = no->pai;
    } else
        novo->pai = NULL;
}

Função para ajustar o ponteiro de um nó da árvore. Usado na função de ajuste.

In [5]:
void ajustaNo(tNo *no, tNo *novo){
    if (no != NULL) {
        if (no->esq == no)
            no->esq = novo;
        else 
            no->dir = novo;
        if (novo != NULL)
           novo->pai = no;
    } else
        novo->pai = NULL;
}

Rotação do nó P à esquerda.

In [6]:
tNo *rotEsquerda(tNo *p){
    tNo *q = p->dir;
    p->dir = q->esq;
    q->pai = p->pai;
    p->pai = q;
    if (q->esq != NULL)
        q->esq->pai = p;
    q->esq = p;
    return q;
}

Rotação do nó P à direita.

In [7]:
tNo* rotDireita(tNo *p){
    tNo *q = p->esq;
    p->esq = q->dir;
    q->pai = p->pai;
    p->pai = q;
    if (q->dir != NULL) 
        q->dir->pai = p;
    q->dir = p;
    return q;
}

Operação de visita do nó. Neste caso imprime a chave e a cor.

In [8]:
 void visita(tNo *no){
    if (no->cor == VERMELHO)
        cout << no->chave << ":"<< "v" <<'.' ;
    else
        cout << no->chave << ":"<< "p" <<'.' ;
}

Percurso da árvore em **PRÉ-ORDEM**.

In [9]:
void preordem(tNo *no){
    if (no != NULL){
        visita(no);
        preordem(no->esq);
        preordem(no->dir);
    }
}

Função para inicialização do nó, com alocação de memória e atribuição de valores para os componentes do nó.

De acordo com as regras de inclusão, novos nós são sempre **vermelhos**.

In [10]:
tNo *criaNo (int chave){
  tNo *n = (tNo *)malloc (sizeof (tNo));
  n->chave = chave;
  n->esq = NULL;
  n->dir = NULL;
  n->pai = NULL; 
  n->cor = VERMELHO;
  return n;
}

Função de ajuste após a inclusão de um nó em uma árvore red black. 

Dado um nó *N* **vermelho**, o ajuste é feito caso a cor do nó pai seja **vermelho**.

A função de ajuste tem 3 casos _simétricos_ para cada lado (3 para a esquerda e 3 para a direita):
1. tio é vermelho
2. tio é preto e o novo nó e filho direito (esq-dir)
3. tio é preto e o novo nó e filho direito (esq-dir)


In [11]:
tNo *ajustaInclusao (tNo *no, tNo *raiz){
    tNo *tio = NULL, *noAsc = NULL; //no para armazenar o tio e o nó da árvore ascendente anterior 
    while (no->pai != NULL && no->pai->cor == VERMELHO){
        if (no->pai == no->pai->pai->esq){//sub arvore da esquerda
            tio = no->pai->pai->dir;
            if (tio != NULL && no->pai->pai->dir->cor == VERMELHO){ // caso 1: tio (direita do avô) é vermelho
                no->pai->cor = PRETO;
                tio->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                no = no->pai->pai;
            } else { //caso 2: tio é preto e o novo nó e filho direito
                 if (no == no->pai->dir){
                     no = no->pai;
                     noAsc = no->pai;
                     noAsc->esq = rotEsquerda(no);
                } //caso 3: tio é preto e novo nó e filho esquerdo
                no->pai->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                noAsc = no->pai->pai->pai; //arvore ascendente
                tNo *novoAvo = rotDireita(no->pai->pai);
                ajustaNo(noAsc, novoAvo);
                if (novoAvo->pai == NULL)
                    raiz = novoAvo;            
            }
        } 
        else { //sub arvore da direita. Mesmo princípio do código acima, porém com ponteiros (esq-dir) "trocados"
            tio = no->pai->pai->esq;
            if (tio != NULL && no->pai->pai->dir->cor == VERMELHO){ //caso 2:
                no->pai->cor = PRETO;
                tio->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                no = no->pai->pai;
            } else {
                 if (no == no->pai->esq){
                     no = no->pai;
                     noAsc = no->pai;
                     noAsc->dir = rotDireita(no);
                }
                no->pai->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                noAsc = no->pai->pai->pai; //arvore ascendente
                tNo *novoAvo = rotEsquerda(no->pai->pai);
                ajustaNo(noAsc, novoAvo);
                if (novoAvo->pai == NULL)
                    raiz = novoAvo;            
            }           
        } 
    }
    raiz->cor = PRETO;
    return raiz;
}

Funcão iterativa para inclusão de novo nó na árvore, dado um nó raiz.

Após a inclusão, chama a função de ajuste.

In [12]:
tNo* inclui (tNo *no, int c, tNo *r){
    tNo *novoNo;
    if (no == NULL) { //inclui na raiz
        novoNo = criaNo(c);
        novoNo->cor = PRETO;
        return novoNo;
    }
    tNo *pai, *raiz = r;
    while (no != NULL)  {
        pai = no;
        if ( c < no->chave)
            no = no->esq;
        else
            no = no->dir;
    }
    novoNo = criaNo(c);
    if (c < pai->chave)
        pai->esq = novoNo;
    else
        pai->dir = novoNo;
    novoNo->pai = pai;
    raiz = ajustaInclusao(novoNo, raiz);
    return raiz;        
}

Função que retorna o nó mínimo de uma sub-árvore. O nó mínimo de uma subárvore é o nó com a menor chave.

In [13]:
tNo *min(tNo *no){
    if (no->esq == NULL) return no;
    else
        return min(no->esq);
}

Função que retorna o nó sucessor de um determinado nó. A chave do nó deverá ser o valor imediatamente superior, em ordem crescente. 

As funções do antecessor e máximo possuem ideia semelhante as do sucessor e mínimo, porém para o lado contrário da árvore.

In [14]:
tNo *sucessor (tNo *no){
    tNo *s = NULL;
    if (no->dir != NULL) return min (no->dir);
    else {
        s = no->pai;
        while (s != NULL && no == s->dir) {
            no = s;
            s = s->pai;
        }        
    }
    return s;
}

Função de ajuste após a exclusão de um nó. A função de ajuste possui 4 casos _simétricos_ para cada sub-árvore (4 para a esquerda e 4 para a direita).

1. O irmão (w) do nó (n) é vermelho 
2. O irmão da direita (d) é preto, e os dois filhos de (d) são pretos
3. irmão da direita (d) é preto, 
    e o filho direito de (d) é preto e o filho esquerdo de (d) é vermelho
4.  irmão da direita (d) é preto, 
    e o filho direito de (d) é vermelho
    
Esta implementação não utiliza explicitamente os **nós nulos**, por isso é necessário realizar checagens adicionais para evitar testes em ponteiros nulos.

In [15]:
tNo *ajustaExclusao (tNo *no, tNo *pai, tNo *raiz){
    tNo *noAsc, *noAux;
    tNo *irmao;
    while (no != raiz && (no == NULL || no->cor == PRETO)){
        if (no == pai->esq) {//sub-árvore da esquerda
            irmao = pai->dir;
            if (irmao->cor == VERMELHO) {//caso 1
                cout << "caso 1 - esq\n";
                irmao->cor = PRETO;
                pai->cor = VERMELHO;
                noAsc = pai->pai;
                noAux = rotEsquerda(pai);
                ajustaNo(noAsc, noAux);
                if (noAsc == NULL) raiz = noAux;
                irmao = pai->dir;
            }
            if ((irmao->esq == NULL || irmao->esq->cor == PRETO) && 
                (irmao->dir == NULL || irmao->dir->cor == PRETO)){ //caso 2
                cout << "caso 2 - esq\n";
                irmao->cor = VERMELHO;
                no = pai;
                pai = no->pai;
            }
            else {
                if ((irmao->esq != NULL && irmao->esq->cor == VERMELHO) &&
                    (irmao->dir == NULL || irmao->dir->cor == PRETO)) { //caso 3
                    cout << "caso 3 - esq\n";
                    irmao->cor = VERMELHO;                    
                    irmao->esq->cor = PRETO;
                    noAsc = irmao->pai;
                    noAsc->dir = rotDireita(irmao);
                    irmao = pai->dir;
                } //caso 4
                cout << "caso 4 - esq\n";
                irmao->cor = pai->cor;
                pai->cor = PRETO;
                if (irmao->dir != NULL)
                    irmao->dir->cor = PRETO;
                noAsc = pai->pai;
                noAux = rotEsquerda(pai);
                if (noAsc != NULL){
                    if (noAsc->esq == pai)
                        noAsc->esq = noAux;
                    else
                        noAsc->dir = noAux;
                }
                noAux->pai = noAsc;
                if (noAsc == NULL) raiz = noAux;
                no = raiz;
            }
        }
        else { //sub-árvore da direita
            irmao = pai->esq;
            if (irmao->cor == VERMELHO) {//caso 1
                cout << "caso 1 - dir\n";
                irmao->cor = PRETO;
                pai->cor = VERMELHO;
                noAsc = pai->pai;
                noAux = rotDireita(pai);
                ajustaNo(noAsc, noAux);
                if (noAsc == NULL) raiz = noAux;
                irmao = pai->esq;
            }
            if ((irmao->dir == NULL || irmao->dir->cor == PRETO) && 
                (irmao->esq == NULL || irmao->esq->cor == PRETO)){ //caso 2
                cout << "caso 2 - dir\n";
                irmao->cor = VERMELHO;
                no = pai;
                pai = no->pai;
            }
            else {
                if ((irmao->dir != NULL && irmao->dir->cor == VERMELHO) && (irmao->esq == NULL ||irmao->esq->cor == PRETO)) { //caso 3
                    cout << "caso 3 - dir\n";
                    irmao->cor = VERMELHO;
                    irmao->dir->cor = PRETO;
                    noAsc = irmao->pai;
                    noAsc->esq = rotEsquerda(irmao);
                    irmao = pai->esq;
                } //caso 4
                cout << "caso 4 - dir\n";
                irmao->cor = pai->cor;
                pai->cor = PRETO;
                if (irmao->esq != NULL)
                    irmao->esq->cor = PRETO;
                noAsc = pai->pai;
                noAux = rotDireita(pai);
                if (noAsc != NULL){
                    if (noAsc->esq == pai)
                        noAsc->esq = noAux;
                    else
                        noAsc->dir = noAux;
                }
                noAux->pai = noAsc;
                if (noAsc == NULL) raiz = noAux; 
                no = raiz;
            }
        }
    }    
    no->cor = PRETO;    
    return raiz;
}

Função que exclui um nó da árvore red black. Usa a regra do sucessor. Retorna a raiz da árvore, pois a árvore poderá ter uma nova raiz, caso seja o nó a ser excluído.

Chama a função de ajuste da exclusão quando o nó excluído é preto, pois altera o balanceamento dos nós pretos (regra 5).

A mesma função poderia ser adaptada para usar a regra do antecessor.

Além do *filhoAjuste*, também é necessário passar como parâmetro o ponteiro *paiAjuste*, devido a não-utilização dos nós-nulos.

In [16]:
tNo* exclui (tNo *no, tNo *raiz) {
    Cor cor_no = no->cor; //armazena a cor do nó
    tNo *s, *novaRaiz = raiz;
    tNo *filhoAjuste = NULL;
    tNo *paiAjuste = NULL;
    if (no->esq == NULL && no->dir == NULL && no->pai == NULL){
        free(no); //exclusão da raiz, último nó
        return NULL;
    }
    if (no->esq == NULL){        
        paiAjuste = no->pai;
        filhoAjuste = no->dir;
        ajustaNoPai(no, no->dir);
        if (no == raiz) novaRaiz = filhoAjuste;
        free (no);
    } else {
        if (no->dir == NULL){
            paiAjuste = no->pai;
            filhoAjuste = no->esq; 
            ajustaNoPai(no, no->esq);
            if (no == raiz) novaRaiz = filhoAjuste;
            free(no);
        }
        else {         
            s = sucessor (no);
            cor_no = s->cor;
            filhoAjuste = s->dir;
            if (s->pai == no)
                paiAjuste = s;
            else
                paiAjuste = s->pai;
            ajustaNoPai(s, filhoAjuste);
            s->esq = no->esq;
            if (s->esq != NULL) 
                s->esq->pai = s;
            if (no->dir != s)
                s->dir = no->dir;
            if (s->dir != NULL)
                s->dir->pai = s;
            ajustaNoPai(no, s);
            s->cor = no->cor; // o sucessor fica com a cor do nó
            if (no == raiz) novaRaiz = s;
            free(no);
        }
    }        
    if (cor_no == PRETO)
        novaRaiz = ajustaExclusao(filhoAjuste, paiAjuste, novaRaiz);
    return novaRaiz;
}

Função que lê um token, separado por espaço, e converte para um número inteiro.

In [17]:
int token_to_num(const char *str, int *indice){    
    char token[100];
    int i = 0;
    while (str[*indice] != '\0' && str[*indice] != ' '){
        token[i] = str[*indice];
        i++;
        (*indice)++;
    }
    token[i] = '\0';
    (*indice)++;    
    return atoi(token);
}

Função que monta uma árvore binária recebendo como entrada uma árvore com parênteses aninhados. Não há suporte a erros de entrada, por isso a árvore passada como parâmetro deve estar correta.

In [18]:
tNo* montaarvore(const char *str){
    tNo *raiz = NULL;
    int i = 0, v =0;
    raiz = inclui(NULL, token_to_num(str,&i), NULL);
    while (str[i]!='\0'){
        raiz = inclui (raiz, token_to_num(str, &i),raiz);                
    }
    return raiz;        
}

Conta o número de nós da árvore.

In [19]:
int contaNos(tNo *no){
    if (no!=NULL) {
        return contaNos(no->esq) + contaNos(no->dir) + 1;
    } else
        return 0;
}

Calcula a altura da árvore. Retorna -1 para considerar a altura da raiz como 0.

In [20]:
int altura (tNo *p) {
    int he, hd;
    if (p == NULL) return -1;
    he = altura (p->esq);
    hd = altura (p->dir);
    if (he > hd)
        return he+1;
    else
        return hd+1;    
}

Busca em árvore binária. Imprime a chave do nó que está sendo visitado para mostrar o caminho percorrido.

In [21]:
tNo *busca(tNo *no, int chave){
    if (no == NULL) return NULL;
    if (no->chave == chave) return no;
    if (chave < no->chave)
        return busca (no->esq, chave);
    else
        return busca (no->dir, chave);
}

Função auxiliar para imprimir a chamada de várias operações.

In [22]:
void imprime(const char *str, tNo *no){
    cout << "||" << str << " ";
    if (no!=NULL) cout << "["<< no->chave<<"]";
    else cout << "não";
    cout << " encontrado\n";
}

O programa abaixo possui diversas sugestões de árvore e operações de inclusão e exclusão, para testar diferentes  casos de exclusão.

In [23]:
void iniciaprograma(){
    tNo *raiz = NULL, *no= NULL;
    int v = 0;

// raiz - unico nó
//char entrada[] = "20\0";
    
// caso 3, 4 - esq
//char entrada[] = "21 5 25 23\0";
//excluir 5

// caso 3, 4 - dir
//char entrada[] = "23 21 25 22\0";
//excluir 25
    
//caso 2 - dir    
char entrada[] = "21 5 22 30\0";
//excluir 30, 22    

//caso 2 - esq
//char entrada[] = "21 5 22 3\0";
//excluir 3, 5 
    
//caso 1, 2 dir
//char entrada[] = "30 20 80 10 25 60 90 5\0";
//excluir 80, 60, 90 

//caso 1, 2 esq
// char entrada[] = "30 20 80 10 25 60 90 100\0";
//excluir 10, 25, 20     
    
//char entrada[] = "30 20 80 10 25 60 90 5 22\0";
//excluir 20 - sucessor    
    
//sucessor vermelho
//char entrada[] = "30 20 80 10 25 60 90 100\0";
//excluir 20 
    
//sucessor raiz, sucessor sem nó direito
//char entrada[] = "30 20 80 10 25 60 90 100\0";
//excluir 30     

    
    raiz=montaarvore(entrada);    
    cout << "\nPercurso em pré-ordem: ";
    preordem(raiz);
    cout << "\nTotal de nós: " << contaNos(raiz);
    cout << "\nAltura da árvore: " << altura(raiz) << "\n";
    cout << "\n";    
    
    v = 30;
    cout << "exclusão :" << v << "\n";
    raiz = exclui( busca(raiz,v), raiz); cout << "\n";
    preordem(raiz); cout << "\n";  
    
    v = 22;
    cout << "exclusão :" << v << "\n";
    raiz = exclui( busca(raiz,v), raiz); cout << "\n";
    preordem(raiz); cout << "\n";      
    
    cout << "\nTotal de nós: " << contaNos(raiz);
    cout << "\nAltura da árvore: " << altura(raiz) << "\n";    
      
}

In [24]:
iniciaprograma();


Percurso em pré-ordem: 23:p.21:p.22:v.25:p.
Total de nós: 4
Altura da árvore: 2

exclusão :25
caso 3 - dir
caso 4 - dir

22:p.21:p.23:p.

Total de nós: 3
Altura da árvore: 1
